In [1]:
import boto3 as boto3
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

df = pd.read_csv("C:/Users/lucab/Documents/FTT/EC/backup_19_03.csv", sep=",", low_memory=False) 
base_juntada_atributes = df 

In [2]:
from flatten_json import (check_if_numbers_are_consecutive, cli, flatten,
                          flatten_preserve_lists, unflatten, unflatten_list)
import pandas as pd
import urllib.request as urllib2
import json
import re

base_juntada_atributes['title_upper'] = base_juntada_atributes['title'].str.upper()

GB_search = []
GBRAM_search = []
for values in base_juntada_atributes['title_upper']:
    
    if re.findall(r"(\d+) GB(?!RAM|\.).*", values):
        GB_search.append(*re.findall(r"(\d+) GB(?!RAM|\.).*", values))
    else:
        GB_search.append(None)
    
    if re.findall(r"(\d+) GB RAM", values):
        GBRAM_search.append(*re.findall(r"(\d+) GB RAM", values))
    else:
        GBRAM_search.append(None)
    
#     print(values)
#     print(*GB_search)
base_juntada_atributes['memory'] = GB_search
base_juntada_atributes['ram_memory'] = GBRAM_search

In [3]:
list_select = ['Linha','Marca','Modelo de GPU','title_upper','Modelo','Modelo do processador','Peso','price','category_file','address.city_name','address.city_id','address.state_name','address.state_id'	,'condition','category_id','permalink', 'memory', 'ram_memory']
base_juntada_atributes_filtrada = base_juntada_atributes[list_select]

base_modelo = base_juntada_atributes_filtrada[base_juntada_atributes_filtrada.price > 1000]

# lista_marcas = ['Apple','Xiaomi', 'Realme', 'Motorola', 'Samsung']

lista_marcas = ['Apple','Xiaomi', 'Motorola', 'Samsung']
base_modelo = base_modelo.drop_duplicates()
base_modelo =  base_modelo.drop_duplicates(subset = 'permalink', keep = 'first')
base_modelo= base_modelo[base_modelo.Marca.isin(lista_marcas)]
# count_result = base_modelo.groupby(['Marca']).size()
# count_result.sort_values(ascending = False)

base_modelo = base_modelo.drop('category_id',1)
base_modelo = base_modelo.drop('title_upper',1)
base_modelo = base_modelo.drop('category_file',1)
base_modelo = base_modelo.drop(['address.city_name','address.city_id','address.state_name'],1)

base_modelo['Peso'] = base_modelo['Peso'].str.replace(' g', "")
base_modelo['Peso'] = base_modelo['Peso'].str.replace(' kg', "") 
base_modelo['Peso'] = pd.to_numeric(base_modelo['Peso'])


In [5]:
colunas_modelos = ['memory', 'ram_memory','Marca', 'Modelo', 'condition','price']
data = base_modelo[colunas_modelos]

In [8]:
predict_teste = data.head(1)
predict_teste

,memory,ram_memory,Marca,Modelo,condition,price
0,64,4,Xiaomi,9 (Global),new,1259.0


In [23]:
unique_count = data.groupby(['Modelo'], as_index=False).agg(uniq_CLIENTCODE =('Modelo',pd.Series.count))


In [25]:
unique_count.sort_values(by=['Modelo'],ascending=False)

,Modelo,uniq_CLIENTCODE
338,serie 5,46
337,pro max,1
336,iPhone XS gold,1
335,iPhone XS Max 256 GG,1
334,iPhone XS Max,69
333,iPhone XS Dual SIM,10
332,iPhone XS,46
331,iPhone XR,131
330,iPhone X,140
329,iPhone SE (2nd Generation),19


In [13]:
data.groupby('Modelo').Modelo.nunique().sort_values(by=['col1'])

Modelo
10                      1
10 Dual SIM             1
10 Lite                 1
10T 5G                  1
10T Lite 5G             1
                       ..
iPhone XS Max           1
iPhone XS Max 256 GG    1
iPhone XS gold          1
pro max                 1
serie 5                 1
Name: Modelo, Length: 339, dtype: int64

In [9]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV

In [10]:
numeric_features = ['memory', 'ram_memory']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Marca','condition','Modelo']
# categorical_features = ['Marca', 'Modelo', 'condition']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])




In [11]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', Ridge())])

In [12]:
from sklearn.model_selection import train_test_split

X = data.drop('price', axis = 1)
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(
                                   X, y, test_size = 0.30)

In [13]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['memory', 'ram_memory']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Marca', 'condition',
                                                   'Modelo'])])),
                ('classifier', Ridge())])

In [18]:
predict_teste = predict_teste.drop(['price'], axis=1)

In [19]:
clf.predict(predict_teste)

array([1355.56527497])

In [95]:
print("model score: %.3f" % clf.score(X_test, y_test))

ValueError: Found unknown categories ['Note10+ 5G Duos', 'Note 10 Pro', 'Note 8T', 'S3 Classic', '8 Pro', '9 Transparent Edition', 'Mix 3', 'A9 (2018)', 'Z3 Play Power Pack & DTV Edition', 'One Fusion', 'X', 'S20+ 5G BTS Edition Duos', 'I PHONE 7 128 gb', 'Nike+', 'MGD83BZ/A', '5 Plus', 'X Play', 'Moto one macro', 'Apple watch 4 40mm', 'DGM 8000', 'GALAXY', 'G7', 'SAMSUNG A11', 'G8 Play', '10 Dual SIM', 'A11', 'S21+ 5G', 'XS MAX', 'G7 Dual SIM', 'Galaxy S9', 'G5 Dual SIM', 'Xcover', 'S7 Edge', 'iPhone XS Max 256 GG', 'Acclaim SCH-R880', 'Poco F2 Pro', 'Mi 10 Coral Green', 'Mi 8 lite', 'Normal', 'Galaxy S10+', 'Z2 Play Dual SIM', 'S21 5G Duos', 'IPhone 11 64g', 'V3 D&G', 'XT2091', 'E4 Plus Dual SIM', 'J7 Prime 2 TV'] in column 2 during transform

In [21]:
import pickle

filename = 'precificacao_4vars.pk1'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
# set up the pipeline
price_pipe = pipe([
    # add a binary variable to indicate missing information for the 2 variables below
    ('one_hot_encoder', OneHotEncoder(variables=['LotFrontage'])),

    # Lasso
    ('lasso', Lasso(random_state=2909, alpha=0.005))

])

In [5]:
json2= {
   "memory": 64,
   "ram_memory": 4,
   "Marca": "Xiaomi",
   "Modelo": "9 (Global)",
   "condition": "new"
}

In [6]:
json_string = json.dumps(json2)


In [7]:
json_string

'{"memory": 64, "ram_memory": 4, "Marca": "Xiaomi", "Modelo": "9 (Global)", "condition": "new"}'

In [19]:
data = json.loads(json_string)
json_normalize(data)

,memory,ram_memory,Marca,Modelo,condition
0,64,4,Xiaomi,9 (Global),new


In [16]:
data

{'memory': 64,
 'ram_memory': 4,
 'Marca': 'Xiaomi',
 'Modelo': '9 (Global)',
 'condition': 'new'}

In [17]:
import json
import pandas as pd
from pandas import json_normalize


In [12]:
train = pd.DataFrame.from_dict(json_string, orient='index')

AttributeError: 'str' object has no attribute 'values'

In [ ]:
dados = request.get_json(force=True)